In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# --- Market Growth Rates (Average Annual) ---
market_rates = {
    "SIP": 0.12,
    "Mutual Fund": 0.10,
    "Stock": 0.15,
    "Gold": 0.08,
    "Silver": 0.06,
    "FD": 0.07,
    "Crypto": 0.20
}

# --- ROI Rating Function ---
def roi_rating(roi):
    if roi < 5:
        return "Poor"
    elif roi < 10:
        return "Average"
    elif roi < 15:
        return "Good"
    else:
        return "Excellent"

# --- Global List to Store Investments ---
investments = []

# --- Function to Add Investment ---
def add_investment():
    name = investment_name.get()
    if name not in market_rates:
        messagebox.showerror("Error", "Unknown investment type! Try SIP, Stock, Gold, etc.")
        return

    try:
        amount = float(investment_amount.get())
        years = int(investment_years.get())
    except ValueError:
        messagebox.showerror("Error", "Please enter valid numbers for amount and years.")
        return

    rate = market_rates[name]
    final_value = amount * ((1 + rate) ** years)
    roi = ((final_value - amount) / amount) * 100

    investments.append((name, amount, final_value, roi))
    update_treeview()

# --- Function to Update Table ---
def update_treeview():
    for row in tree.get_children():
        tree.delete(row)
    
    for i, (name, amount, final, roi) in enumerate(investments, start=1):
        tree.insert("", "end", values=(i, name, f"{amount:.2f}", f"{final:.2f}", f"{roi:.2f}%", roi_rating(roi)))

# --- Function to Open New Dashboard Window ---
def open_dashboard():
    if not investments:
        messagebox.showerror("Error", "No data to display.")
        return
    
    # New Window for Charts
    dash = tk.Toplevel(window)
    dash.title("ROI Dashboard Charts")
    dash.geometry("1100x700")
    dash.configure(bg="#F8F9FA")

    # Scrollable Canvas for Charts
    canvas_chart = tk.Canvas(dash, bg="#F8F9FA")
    scroll_y = tk.Scrollbar(dash, orient="vertical", command=canvas_chart.yview)
    scroll_x = tk.Scrollbar(dash, orient="horizontal", command=canvas_chart.xview)
    
    scrollable_frame = tk.Frame(canvas_chart, bg="#F8F9FA")
    
    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas_chart.configure(scrollregion=canvas_chart.bbox("all"))
    )
    
    canvas_chart.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas_chart.configure(yscrollcommand=scroll_y.set, xscrollcommand=scroll_x.set)

    canvas_chart.pack(side="left", fill="both", expand=True)
    scroll_y.pack(side="right", fill="y")
    scroll_x.pack(side="bottom", fill="x")

    # Create Charts
    names = [inv[0] for inv in investments]
    rois = [inv[3] for inv in investments]
    amounts = [inv[1] for inv in investments]
    finals = [inv[2] for inv in investments]
    
    # Slightly reduced chart size
    fig, axs = plt.subplots(2, 2, figsize=(11, 7))
    fig.suptitle("ROI Tracker Dashboard", fontsize=15, color="#2DACA7", weight="bold")

    # --- Bar Chart for ROI % ---
    axs[0, 0].bar(names, rois, color="#2DACA7", edgecolor="black", width=0.5)
    axs[0, 0].set_title("ROI % Comparison", fontsize=10)
    axs[0, 0].set_ylabel("ROI (%)", fontsize=9)
    axs[0, 0].grid(axis="y", linestyle="--", alpha=0.5)

    # --- Line Chart for Growth ---
    axs[0, 1].plot(names, amounts, 'o--g', label="Initial", linewidth=1.8)
    axs[0, 1].plot(names, finals, 's--r', label="Final", linewidth=1.8)
    axs[0, 1].set_title("Investment Growth Over Time", fontsize=10)
    axs[0, 1].legend(fontsize=8)
    axs[0, 1].grid(True, linestyle="--", alpha=0.5)

    # --- Pie Chart: Investment Distribution ---
    axs[1, 0].pie(amounts, labels=names, autopct="%1.1f%%", startangle=140, textprops={'fontsize':8})
    axs[1, 0].set_title("Initial Investment Distribution", fontsize=10)

    # --- Scatter Chart: ROI vs Final Value ---
    axs[1, 1].scatter(rois, finals, color="#FF7F0E", s=80)
    axs[1, 1].set_title("ROI vs Final Value", fontsize=10)
    axs[1, 1].set_xlabel("ROI (%)", fontsize=9)
    axs[1, 1].set_ylabel("Final Value (₹)", fontsize=9)
    axs[1, 1].grid(True, linestyle="--", alpha=0.5)

    fig.tight_layout(pad=3.0)

    # --- Embed Chart in Scrollable Frame ---
    canvas = FigureCanvasTkAgg(fig, master=scrollable_frame)
    canvas.draw()
    canvas_widget = canvas.get_tk_widget()
    canvas_widget.pack(pady=25, fill="both", expand=True)

# --- Main Window Setup ---
window = tk.Tk()
window.title("ROI Tracker - Main Dashboard")
window.geometry("1250x750")
window.configure(bg="#F8F9FA")

# --- Header ---
tk.Label(window, text="ROI TRACKER DASHBOARD", font=("Arial", 22, "bold"), fg="#2DACA7", bg="#F8F9FA").grid(row=0, column=0, columnspan=4, pady=20)

# --- Input Section ---
tk.Label(window, text="Investment Type:", bg="#F8F9FA", font=("Arial", 12)).grid(row=1, column=0, padx=10, pady=10, sticky="e")
investment_name = ttk.Combobox(window, values=list(market_rates.keys()), width=25)
investment_name.grid(row=1, column=1)

tk.Label(window, text="Initial Amount (₹):", bg="#F8F9FA", font=("Arial", 12)).grid(row=2, column=0, sticky="e")
investment_amount = tk.Entry(window, width=28)
investment_amount.grid(row=2, column=1)

tk.Label(window, text="Duration (Years):", bg="#F8F9FA", font=("Arial", 12)).grid(row=3, column=0, sticky="e")
investment_years = tk.Entry(window, width=28)
investment_years.grid(row=3, column=1)

# --- Buttons ---
tk.Button(window, text="Add Investment", command=add_investment, bg="#2DACA7", fg="white", font=("Arial", 11, "bold")).grid(row=4, column=0, columnspan=2, pady=10)
tk.Button(window, text="Show Charts Dashboard", command=open_dashboard, bg="#007BFF", fg="white", font=("Arial", 11, "bold")).grid(row=4, column=2, columnspan=2, pady=10)

# --- Table Section ---
columns = ("#", "Investment", "Initial (₹)", "Final (₹)", "ROI (%)", "Rating")
tree = ttk.Treeview(window, columns=columns, show="headings", height=8)

column_widths = [60, 150, 160, 160, 100, 120]
for col, width in zip(columns, column_widths):
    tree.heading(col, text=col)
    tree.column(col, anchor="center", width=width)

tree.grid(row=5, column=0, columnspan=4, padx=30, pady=15)

window.mainloop()